### 싱글모델로 분류

In [1]:

import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data import Subset
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import cv2
import random
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [2]:
with open('../configs/NMC.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

{'DEVICE': 'cuda:0', 'SAVE_DIR': 'output', 'MODEL': {'NAME': 'EfficientNetV2MModelMulti', 'BACKBONE': 'EfficientNetV2', 'PRETRAINED': '/workspace/jhmoon/nmc_2024/checkpoints/pretrained/tf_efficientnetv2_m_weights.pth', 'UNFREEZE': 'full', 'VERSION': "384_32_loss'"}, 'DATASET': {'NAME': 'NMCDataset', 'ROOT': '/data/nmc/processed_image', 'TRAIN_RATIO': 0.7, 'VALID_RATIO': 0.15, 'TEST_RATIO': 0.15}, 'TRAIN': {'IMAGE_SIZE': [384, 384], 'BATCH_SIZE': 32, 'EPOCHS': 100, 'EVAL_INTERVAL': 1, 'AMP': False, 'DDP': False}, 'LOSS': {'NAME': 'BCEWithLogitsLoss', 'CLS_WEIGHTS': False}, 'OPTIMIZER': {'NAME': 'adamw', 'LR': 0.1, 'WEIGHT_DECAY': 0.01}, 'SCHEDULER': {'NAME': 'warmuppolylr', 'POWER': 0.9, 'WARMUP': 10, 'WARMUP_RATIO': 0.1}, 'EVAL': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.NMC.pth', 'IMAGE_SIZE': [384, 384]}, 'TEST': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.NMC.pth', 'FILE': 'assests/ade', 'IMAGE_SIZE': [384, 384], 'OVERLAY': True}}


In [3]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

In [4]:
def get_train_augmentation(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_test_transform(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [5]:
class MultiTargetBalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size, target_classes):
        self.dataset = dataset
        self.batch_size = batch_size
        self.target_classes = target_classes
        
        # 데이터셋에서 레이블 추출
        if hasattr(dataset, 'labels'):
            self.labels = dataset.labels
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        elif hasattr(dataset, 'targets'):
            self.labels = dataset.targets
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        else:
            try:
                self.labels = [sample[1] for sample in dataset]
                if isinstance(self.labels[0], np.ndarray):
                    self.labels = torch.from_numpy(np.array(self.labels))
                else:
                    self.labels = torch.tensor(self.labels)
            except:
                raise ValueError("Cannot access labels from dataset")
        
        # 각 타겟 클래스와 나머지 클래스의 인덱스 저장
        self.target_indices = {}
        for target in target_classes:
            if len(self.labels.shape) > 1:
                self.target_indices[target] = torch.where(self.labels[:, target] == 1)[0]
            else:
                self.target_indices[target] = torch.where(self.labels == target)[0]
        
        # 나머지 클래스의 인덱스 저장
        if len(self.labels.shape) > 1:
            self.other_indices = torch.where(
                torch.sum(self.labels[:, target_classes], dim=1) == 0)[0]
        else:
            mask = torch.ones_like(self.labels, dtype=torch.bool)
            for target in target_classes:
                mask &= (self.labels != target)
            self.other_indices = torch.where(mask)[0]
        
        # 각 그룹당 샘플 수 계산
        n_groups = len(target_classes) + 1  # 타겟 클래스들 + 나머지
        self.samples_per_group = batch_size // n_groups
        
        self.n_batches = len(self.dataset) // batch_size
        if len(self.dataset) % batch_size != 0:
            self.n_batches += 1
    
    def __iter__(self):
        for _ in range(self.n_batches):
            batch_indices = []
            
            # 각 타겟 클래스에서 샘플링
            for target in self.target_classes:
                target_selected = self.target_indices[target][
                    torch.randint(len(self.target_indices[target]), 
                                (self.samples_per_group,))
                ]
                batch_indices.extend(target_selected.tolist())
            
            # 나머지 클래스들에서 샘플링
            other_selected = self.other_indices[
                torch.randint(len(self.other_indices), 
                            (self.samples_per_group,))
            ]
            batch_indices.extend(other_selected.tolist())
            
            # 배치 셔플
            random.shuffle(batch_indices)
            
            # 배치 크기에 맞게 자르기 (나누어 떨어지지 않는 경우 처리)
            if len(batch_indices) > self.batch_size:
                batch_indices = batch_indices[:self.batch_size]
            
            yield batch_indices
    
    def __len__(self):
        return self.n_batches

In [6]:
def train_epoch(model, dataloader, criterion, optimizer, scaler, device, target_label_idx):
    model.train()
    running_loss = 0.0
    num_targets = len(target_label_idx)

    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        
        if num_targets == 1:
            # 단일 레이블 케이스
            target_labels = labels[:, target_label_idx].to(device)
            
            optimizer.zero_grad()
            
            with autocast(enabled=True):
                outputs = model(images)
                # 차원을 맞춰줌
                outputs = outputs.view(-1)  # or outputs.squeeze()
                target_labels = target_labels.view(-1)  # or target_labels.squeeze()
                loss = criterion(outputs, target_labels)
        else:
            # 다중 레이블 케이스
            target_labels = labels[:, target_label_idx].to(device)
            
            optimizer.zero_grad()
            
            with autocast(enabled=True):
                outputs = model(images)
                loss = criterion(outputs, target_labels)

        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

In [7]:
def evaluate(model, dataloader, device, target_label_idx):
   model.eval()
   all_preds = []
   all_labels = []
   num_targets = len(target_label_idx)
   
   with torch.no_grad():
       for images, labels in tqdm(dataloader, desc="Evaluating"):
           images = images.to(device)
           
           if num_targets == 1:
               # 단일 레이블 케이스
               target_labels = labels[:, target_label_idx].to(device)
               outputs = model(images)
               
               # 차원 처리
               if len(outputs.shape) == 2:
                   outputs = outputs.squeeze(1)
               
               preds = (torch.sigmoid(outputs) > 0.5).float()
               
               all_preds.append(preds.cpu().numpy().reshape(-1))
               all_labels.append(target_labels.cpu().numpy().reshape(-1))
           
           else:
               # 다중 레이블 케이스
               target_labels = labels[:, target_label_idx].to(device)
               outputs = model(images)
               
               # 각 레이블에 대한 예측
               preds = (torch.sigmoid(outputs) > 0.5).float()
               
               all_preds.append(preds.cpu().numpy())
               all_labels.append(target_labels.cpu().numpy())
   
   # numpy array로 변환
   all_preds = np.concatenate(all_preds)
   all_labels = np.concatenate(all_labels)
   
   if num_targets == 1:
       # 단일 레이블 메트릭
       f1 = f1_score(all_labels, all_preds, average='binary')
       accuracy = accuracy_score(all_labels, all_preds)
       precision = precision_score(all_labels, all_preds)
       recall = recall_score(all_labels, all_preds)
       
       return f1, accuracy, precision, recall
   else:
       # 다중 레이블 메트릭
       f1 = f1_score(all_labels, all_preds, average='macro')
       accuracy = accuracy_score(all_labels, all_preds)
       precision = precision_score(all_labels, all_preds, average='macro')
       recall = recall_score(all_labels, all_preds, average='macro')
       
       return f1, accuracy, precision, recall

In [8]:
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, scaler, device, epochs, target_label_idx):
    best_f1 = 0.0
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    num_targets = len(target_label_idx)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, device, target_label_idx)
        
        if num_targets == 1:
            val_f1, val_acc, val_prec, val_rec = evaluate(model, val_loader, device, target_label_idx)
            
            print(f"Training Loss: {train_loss:.4f}")
            print(f"Validation Metrics:")
            print(f"  F1 Score: {val_f1:.4f}")
            print(f"  Accuracy: {val_acc:.4f}") 
            print(f"  Precision: {val_prec:.4f}")
            print(f"  Recall: {val_rec:.4f}")
            
            scheduler.step(val_f1)
            
            if val_f1 > best_f1:
                best_f1 = val_f1
                torch.save(model.state_dict(), f'model/singlelabel/best_model_label_{target_label_idx[0]}_nmc_cnn.pth')
                print("New best model saved!")
        else:
            # 모든 메트릭을 받아서 f1만 사용
            val_f1, val_acc, val_prec, val_rec = evaluate(model, val_loader, device, target_label_idx)
            
            print(f"Training Loss: {train_loss:.4f}")
            print(f"Validation Metrics:")
            print(f"  Macro F1 Score: {val_f1:.4f}")
            print(f"  Macro Accuracy: {val_acc:.4f}")
            print(f"  Macro Precision: {val_prec:.4f}")
            print(f"  Macro Recall: {val_rec:.4f}")
            
            scheduler.step(val_f1)
            
            if val_f1 > best_f1:
                best_f1 = val_f1
                torch.save(model.state_dict(), f'model/singlelabel/best_model_labels_{"-".join(map(str,target_label_idx))}_nmc_cnn.pth')
                print("New best model saved!")
        
        early_stopping(val_f1)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
            
        print()
    
    return best_f1

In [9]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
device = "cuda:1"
print("device : ", device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

image_size = [256,256]
image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
train_transform = get_train_augmentation(image_size)
val_test_transform = get_val_test_transform(image_size)
batch_size = 32


dataset = eval(dataset_cfg['NAME'])(
    dataset_cfg['ROOT'] + '/cropped_images_1424x1648',
    dataset_cfg['TRAIN_RATIO'],
    dataset_cfg['VALID_RATIO'],
    dataset_cfg['TEST_RATIO'],
    transform=None
)
trainset, valset, testset = dataset.get_splits()
valset.transform = val_test_transform
testset.transform = val_test_transform

# trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)
    

device :  cuda:1
/data/nmc/processed_image/cropped_images_1424x1648
(0,)               1935
(3,)                542
(1, 2, 3)           532
(1, 2)              286
(2,)                233
(1, 2, 3, 4)        190
(2, 3)              163
(1,)                155
(4,)                 47
(1, 3)               31
(1, 2, 4)            27
(3, 4)               22
(1, 2, 3, 4, 5)      11
(5,)                  9
(2, 3, 4)             9
(1, 4)                9
(1, 2, 3, 5)          8
(1, 2, 5)             7
(2, 4)                7
(1, 2, 3, 5, 6)       5
(1, 2, 3, 6)          4
(1, 3, 4)             2
(1, 3, 6)             1
(6,)                  1
(1, 2, 6)             1
(1, 2, 3, 4, 6)       1
Name: label, dtype: int64
train size: 4238
(0,)               415
(3,)               118
(1, 2, 3)          113
(1, 2)              65
(2,)                50
(1, 2, 3, 4)        46
(2, 3)              35
(1,)                32
(4,)                13
(1, 3)               7
(5,)                 4
(3, 4)      

In [12]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2
import os
def denormalize(tensor):
   """Denormalize the image tensor"""
   mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
   std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
   return tensor * std + mean

def occlusion_sensitivity_map(model, image, target_label_idx, patch_size=14, stride=7, device='cuda'):
    model.eval()
    height, width = image.shape[-2:]
    sensitivity_map = torch.zeros((height, width), device=device)
    
    # 타겟 인덱스 처리 수정
    if isinstance(target_label_idx, list):
        target_idx = target_label_idx[0]
    else:
        target_idx = target_label_idx
        
    with torch.no_grad():
        base_output = model(image.unsqueeze(0))
        base_pred = torch.sigmoid(base_output)[0, target_idx].item()
   
    # 패치 단위로 가리며 예측값 변화 측정
    for y in range(0, height-patch_size+1, stride):
        for x in range(0, width-patch_size+1, stride):
            occluded = image.clone()
            occluded[..., y:y+patch_size, x:x+patch_size] = 0
            
            with torch.no_grad():
                output = model(occluded.unsqueeze(0))
                pred = torch.sigmoid(output)[0, target_label_idx].item()
            
            diff = abs(base_pred - pred)
            sensitivity_map[y:y+patch_size, x:x+patch_size] += diff
            
    # 정규화
    sensitivity_map = (sensitivity_map - sensitivity_map.min()) / (sensitivity_map.max() - sensitivity_map.min() + 1e-8)
    return sensitivity_map.cpu().numpy()

def create_comparison_image(original_img, nmc_heatmap, aptos_heatmap, label_info, nmc_correct, aptos_correct, save_path):
   # Denormalize the original image
   orig_img = denormalize(torch.from_numpy(original_img)).numpy()
   orig_img = np.clip(orig_img.transpose(1, 2, 0), 0, 1)
   orig_img = np.uint8(orig_img * 255)
   
   # Create heatmap overlays
   nmc_heatmap_rgb = np.uint8(255 * nmc_heatmap)
   nmc_heatmap_rgb = cv2.applyColorMap(nmc_heatmap_rgb, cv2.COLORMAP_JET)
   nmc_superimposed = cv2.addWeighted(orig_img, 0.6, nmc_heatmap_rgb, 0.4, 0)
   
   aptos_heatmap_rgb = np.uint8(255 * aptos_heatmap)
   aptos_heatmap_rgb = cv2.applyColorMap(aptos_heatmap_rgb, cv2.COLORMAP_JET)
   aptos_superimposed = cv2.addWeighted(orig_img, 0.6, aptos_heatmap_rgb, 0.4, 0)
   
   # Create a white background for label info
   height, width = orig_img.shape[:2]
   info_height = height // 2  # 라벨 정보 영역의 높이를 절반으로
   info_bg = np.ones((info_height, width * 3, 3), dtype=np.uint8) * 255  # 3배 너비
   
   # Add text to info background
   font = cv2.FONT_HERSHEY_SIMPLEX
   font_scale = 0.5  # 글씨 크기 축소
   thickness = 1
   color = (0, 0, 0)  # Black color
   
   # Add label information
   y_offset = 20
   for line in label_info:
       text_size = cv2.getTextSize(line, font, font_scale, thickness)[0]
       x = (info_bg.shape[1] - text_size[0]) // 2
       y = y_offset
       cv2.putText(info_bg, line, (x, y), font, font_scale, color, thickness)
       y_offset += 20
   
   # Create result texts
   nmc_result = "NMC: Correct" if nmc_correct else "NMC: Incorrect"
   aptos_result = "APTOS: Correct" if aptos_correct else "APTOS: Incorrect"
   
   # Add model results with color coding
   text_size = cv2.getTextSize(nmc_result, font, font_scale, thickness)[0]
   x = width + (width - text_size[0]) // 2
   y = height // 2 - 20
   color = (0, 255, 0) if nmc_correct else (0, 0, 255)
   cv2.putText(info_bg, nmc_result, (x, y), font, font_scale, color, thickness)
   
   text_size = cv2.getTextSize(aptos_result, font, font_scale, thickness)[0]
   x = 2 * width + (width - text_size[0]) // 2
   color = (0, 255, 0) if aptos_correct else (0, 0, 255)
   cv2.putText(info_bg, aptos_result, (x, y), font, font_scale, color, thickness)
   
   # Concatenate images horizontally
   images_row = np.concatenate([orig_img, nmc_superimposed, aptos_superimposed], axis=1)
   
   # Concatenate info and images vertically
   final_image = np.concatenate([info_bg, images_row], axis=0)
   
   # Save the final image
   cv2.imwrite(save_path, cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR))

def compare_and_save_gradcam(nmc_model, aptos_model, dataloader, nmc_label_idx, aptos_label_idx, device, save_dir='osm_results'):
   nmc_model.eval()
   aptos_model.eval()
   
   both_correct = []
   only_nmc_correct = []
   only_aptos_correct = []
   both_wrong = []
   
   save_path = f"{save_dir}/comparison/label_{'-'.join(map(str, nmc_label_idx))}_vs_{aptos_label_idx}"
   os.makedirs(save_path, exist_ok=True)
   
   with torch.no_grad():
       for batch_idx, (images, labels) in enumerate(dataloader):
           images = images.to(device)
           labels = labels.to(device)
           
           # APTOS model predictions (기준)
           aptos_outputs = aptos_model(images)
           aptos_predictions = (torch.sigmoid(aptos_outputs) > 0.5).squeeze()
           aptos_raw_preds = torch.sigmoid(aptos_outputs).squeeze()
           aptos_targets = labels[:, aptos_label_idx]
           
           # NMC model predictions
           nmc_outputs = nmc_model(images)
           nmc_predictions = (torch.sigmoid(nmc_outputs) > 0.5)
           nmc_raw_preds = torch.sigmoid(nmc_outputs)
           
           # APTOS 라벨을 기준으로 NMC 예측 평가
           nmc_correct = torch.where(
               aptos_targets == 1,
               torch.all(nmc_predictions == 1, dim=1),  # APTOS가 1일 때
               torch.all(nmc_predictions == 0, dim=1)   # APTOS가 0일 때
           )
           
           # Handle batch size 1 case
           if len(images) == 1:
               aptos_predictions = aptos_predictions.unsqueeze(0)
               aptos_raw_preds = aptos_raw_preds.unsqueeze(0)
               aptos_targets = aptos_targets.unsqueeze(0)
               nmc_correct = nmc_correct.unsqueeze(0)
           
           for i in range(len(images)):
               try:
                   aptos_correct = (aptos_predictions[i].item() == aptos_targets[i].item())
                   nmc_is_correct = nmc_correct[i].item()
                   
                   sample_info = {
                       'image': images[i],
                       'aptos_target': aptos_targets[i].item(),
                       'nmc_preds': nmc_raw_preds[i].cpu().numpy(),
                       'aptos_pred': aptos_raw_preds[i].item()
                   }
                   
                   # Categorize samples
                   if aptos_correct and nmc_is_correct and len(both_correct) < 3:
                       both_correct.append(sample_info)
                   elif not aptos_correct and nmc_is_correct and len(only_nmc_correct) < 3:
                       only_nmc_correct.append(sample_info)
                   elif aptos_correct and not nmc_is_correct and len(only_aptos_correct) < 3:
                       only_aptos_correct.append(sample_info)
                   elif not aptos_correct and not nmc_is_correct and len(both_wrong) < 3:
                       both_wrong.append(sample_info)
               
               except Exception as e:
                   print(f"Error processing sample {i} in batch {batch_idx}: {e}")
                   continue
               
               if (len(both_correct) >= 3 and 
                   len(only_nmc_correct) >= 3 and 
                   len(only_aptos_correct) >= 3 and 
                   len(both_wrong) >= 3):
                   break
                   
           if (len(both_correct) >= 3 and 
               len(only_nmc_correct) >= 3 and 
               len(only_aptos_correct) >= 3 and 
               len(both_wrong) >= 3):
               break
   
   print(f"\nSamples collected:")
   print(f"Both correct: {len(both_correct)}")
   print(f"Only NMC correct: {len(only_nmc_correct)}")
   print(f"Only APTOS correct: {len(only_aptos_correct)}")
   print(f"Both wrong: {len(both_wrong)}")
   
   # Save GradCAM visualizations for all categories
   for category, samples, category_name in [
       (both_correct, "both_correct", "Both Correct"),
       (only_nmc_correct, "only_nmc", "Only NMC Correct"),
       (only_aptos_correct, "only_aptos", "Only APTOS Correct"),
       (both_wrong, "both_wrong", "Both Wrong")
   ]:
       for idx, sample in enumerate(category):
           try:
               nmc_heatmap = occlusion_sensitivity_map(nmc_model, sample['image'], nmc_label_idx[0], device=device)
               aptos_heatmap = occlusion_sensitivity_map(aptos_model, sample['image'], 0, device=device)

               
               # Prepare label info
               nmc_pred_str = np.array2string(sample['nmc_preds'], precision=3)
               aptos_pred_str = f"{sample['aptos_pred']:.3f}"
               aptos_target_str = f"{int(sample['aptos_target'])}"
               
               nmc_is_correct = torch.all(torch.tensor(sample['nmc_preds'] > 0.5) == (sample['aptos_target'] == 1))
               aptos_is_correct = (sample['aptos_pred'] > 0.5) == sample['aptos_target']
               
               label_info = [
                   f"Category: {category_name}",
                   f"NMC Labels: {nmc_label_idx}, APTOS Label: {aptos_label_idx}",
                   f"APTOS - True: {aptos_target_str}, Pred: {aptos_pred_str}",
                   f"NMC Predictions: {nmc_pred_str}"
               ]
               
               # Save combined result
               save_name = os.path.join(save_path, f'{category_name.lower().replace(" ", "_")}_{idx}.png')
               create_comparison_image(
                   sample['image'].cpu().numpy(),
                   nmc_heatmap,
                   aptos_heatmap,
                   label_info,
                   nmc_is_correct,
                   aptos_is_correct,
                   save_name
               )
               
           except Exception as e:
               print(f"Error processing {category_name} sample {idx}: {e}")
               print(f"Sample info: {sample}")
               continue

In [13]:
# Main execution
nmc_labels = [[0],[2],[1],[1,2],[5,6]]
aptos_labels = [0,1,2,3,4]  # 각각 대응되는 APTOS 라벨

for idx, nmc_label_idx in enumerate(nmc_labels):
   aptos_label_idx = aptos_labels[idx]
   
   print(f"\nProcessing NMC label {nmc_label_idx} and APTOS label {aptos_label_idx}")
   
   # Load NMC model
   nmc_model = models.efficientnet_v2_m(pretrained=True)
   num_ftrs = nmc_model.classifier[1].in_features
   nmc_model.classifier = nn.Sequential(
       nn.BatchNorm1d(num_ftrs),
       nn.Linear(num_ftrs, len(nmc_label_idx))
   )
   nmc_model = nmc_model.to(device)
   
   if len(nmc_label_idx)==1:
       nmc_model.load_state_dict(torch.load(f'model/singlelabel/best_model_label_{nmc_label_idx[0]}_nmc_cnn.pth'))
   else:
       nmc_model.load_state_dict(torch.load(f'model/singlelabel/best_model_labels_{"-".join(map(str,nmc_label_idx))}_nmc_cnn.pth'))
   
   # Load APTOS model
   aptos_model = models.efficientnet_v2_m(pretrained=True)
   aptos_model.classifier = nn.Sequential(
       nn.BatchNorm1d(num_ftrs),
       nn.Linear(num_ftrs, 1)
   )
   aptos_model = aptos_model.to(device)
   aptos_model.load_state_dict(torch.load(f'model/singlelabel/best_model_label_{aptos_label_idx}_aptos_cnn.pth'))
   
   # Compare and save results
   compare_and_save_gradcam(nmc_model, aptos_model, testloader, nmc_label_idx, aptos_label_idx, device)


Processing NMC label [0] and APTOS label 0

Samples collected:
Both correct: 3
Only NMC correct: 3
Only APTOS correct: 3
Both wrong: 3

Processing NMC label [2] and APTOS label 1

Samples collected:
Both correct: 3
Only NMC correct: 3
Only APTOS correct: 3
Both wrong: 3
Error processing Both Correct sample 0: index 2 is out of bounds for dimension 1 with size 1
Sample info: {'image': tensor([[[-2.0837, -2.0837, -2.1008,  ..., -2.0837, -2.0837, -2.1008],
         [-2.0494, -2.1008, -2.1008,  ..., -2.0837, -2.0837, -2.1008],
         [-2.0837, -2.1008, -2.1008,  ..., -2.1008, -2.0837, -2.1008],
         ...,
         [-2.0665, -2.0665, -2.0665,  ..., -2.0494, -2.0665, -2.0837],
         [-2.0665, -2.0665, -2.0665,  ..., -2.0494, -2.0665, -2.0837],
         [-2.0665, -2.0494, -2.0837,  ..., -2.0665, -2.0665, -2.0837]],

        [[-2.0007, -2.0357, -2.0182,  ..., -2.0357, -2.0007, -2.0182],
         [-2.0357, -2.0182, -2.0182,  ..., -2.0182, -2.0357, -2.0182],
         [-2.0357, -2.0182, 